# 相机内参标定
## 内参标定原理
## opencv中标定api
## 畸变矫正
### undistort
### remap
## 标定结果评估(重新投影误差)

In [11]:
import numpy as np
import cv2 as cv
import glob
# 终止条件, 当达到30次迭代或者精度epsilon=0.001时停止迭代
# TERM_CRITERIA_EPS是指定精度，TERM_CRITERIA_MAX_ITER是指定迭代次数
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# 准备对象点， 如 (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# 用于存储所有图像的对象点和图像点的数组。
objpoints = [] # 真实世界中的3d点
imgpoints = [] # 图像中的2d点
images = glob.glob('../data/left0*.jpg')#glob.glob()返回所有匹配的文件路径列表。
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # 找到棋盘角落
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # 如果找到，添加对象点，图像点（细化之后）
    if ret == True:
        objpoints.append(objp)#将世界坐标系的坐标放入objpoints中
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)#细化角点坐标
        imgpoints.append(corners)#将图像坐标系的坐标放入imgpoints中
        # 绘制并显示拐角
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        #cv.imshow('img', img)
        cv.waitKey(500)
# 标定,返回值分别是：ret:标定结果；mtx:相机内参；dist:畸变系数；rvecs:旋转向量；tvecs:平移向量
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None) 
# print("ret:",ret)
#print("mtx:\n",mtx)
# print("dist:\n",dist)
# print("rvecs:\n",rvecs)
# print("tvecs:\n",tvecs)
img = cv.imread('../data/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
# print(newcameramtx)
 # undistort
dst1 = cv.undistort(img, mtx, dist, None, newcameramtx)
# 剪裁图像
x, y, w, h = roi
dst1 = dst1[y:y+h, x:x+w]

# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst2 = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# 裁剪图像
x, y, w, h = roi
dst2 = dst2[y:y+h, x:x+w]

#两张图片合并在一起显示,两张像素大小不一致,需要调整大小
#调整大小
dst1 = cv.resize(dst1,(640,480))
dst2 = cv.resize(dst2,(640,480))
hmerge = np.concatenate((dst1, dst2), axis=1)

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

cv.imshow('hmerge', hmerge)
cv.waitKey(0)
cv.destroyAllWindows()


total error: 0.023287284862136742


>objp = np.zeros((6*7,3), np.float32)  
>objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)  

让我们来解释一下objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)这段代码的含义。
首先，让我们从右往左逐步解析这段代码：

1. np.mgrid[0:7,0:6]：np.mgrid是一个用于生成多维坐标网格的函数。在这里，我们使用np.mgrid[0:7,0:6]生成一个2D坐标网格，其中第一维的取值范围是0到6，第二维的取值范围是0到5。这将生成一个形状为(7, 6, 2)的数组，其中每个元素都是一个包含两个坐标值的数组。

2. .T：.T是数组的转置操作符。它将对先前生成的数组进行转置，使得第一维和第二维的坐标值交换位置。转置之后，数组的形状变为(6, 7, 2)。

3. .reshape(-1, 2)：.reshape()方法用于重新塑造数组的形状。在这里，我们将数组重新塑造为一个两列的数组，而不关心行数。通过使用-1作为参数，reshape()方法将根据数组的总元素数量自动计算行数。在这个示例中，数组的总元素数量是42（6行 x 7列），因此结果将是一个形状为(42, 2)的数组。

现在，我们来看objp[:,:2] = ...部分：

objp[:,:2]：表示对objp数组的切片操作，选取所有行的前两列。
= np.mgrid[0:7,0:6].T.reshape(-1,2)：将右侧计算的数组赋值给切片所选的部分。
综上所述，这段代码的目的是将一个表示棋盘格对象点的数组objp初始化为一个形状为(42, 3)的二维数组，其中前两列是棋盘格的x和y坐标（从0到6，共7列），而第三列都是0。这样，我们就得到了一组在真实世界中的棋盘格角点的3D坐标。